In this HW we had to implement regression and gradient descent ourselves in python. I've done this before in Golang which tended to be faster (check [newborn repo](https://github.com/elahe-dastan/newborn) if intersted) because Golang is faster than python I didn't do any test but if you are about to do something similar pay attention that the vectorization used in python like what numpy does most of the time boosts the speed which we may not have in other languages.<br/>
knowing the simple math behind these algorithms is enough for implementation:<br/>
[check here for the math behind](https://elahe-dastan.github.io/newborn/)

In [2]:
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from math import sqrt

class regression:
    GRADIENT_DESCENT = 'gradient_descent'
    NORMAL_EQUATION = 'normal_equation'
    def __init__(self, degree: int, method=GRADIENT_DESCENT):
        self.degree = degree
        # Example:
        # if the training dataset has feature x and the degree is 2 it uses both x and x^2 for training.
        self.method = method
        self.coefficients = None
        self.steps = np.array([])
        
        self.train_error_mse = np.array([])
        self.test_error_mse = np.array([])
        
        self.train_error_mae = np.array([])
        self.test_error_mae = np.array([])
        
        self.train_error_rmse = np.array([])
        self.test_error_rmse = np.array([])
    
    def initialization(self, x):
        features_num = x.shape[1]
        
        for i in range(self.degree-1):
            x = np.c_[x, np.multiply(x[:, 0:features_num], x[:, i*features_num:(i+1)*features_num])]
        
        # putting a vector of one in the first column for bias
        x = np.c_[np.ones(len(x)), x]
        
        return x

    def train(self, x: np.array, y: np.array, learning_rate=0.1, iteration=10000, lam=0):
        self.learning_rate = learning_rate
        self.iteration = iteration
        features_num = x.shape[1]
        # plus 1 is for considering bias
        self.coefficients = np.random.random((self.degree * features_num) + 1)
        
        if self.method == self.GRADIENT_DESCENT:
            self.__train_gradient_descent(x, y, learning_rate, iteration)
        elif self.method == self.NORMAL_EQUATION:
            self.__normal_equation(x, y, lam)
        else:
            print('I only know the methods {:s} and {:s}'.format(self.GRADIENT_DESCENT, self.NORMAL_EQUATION))
        
    def __train_gradient_descent(self, x: np.array, y: np.array, learning_rate: float, iteration: int):
        x = self.initialization(x)
        
        for i in range(iteration):
            self.gradient_descent(x, y, learning_rate)
            return
    
    def gradient_descent(self, x, y, learning_rate: float):
        m = len(x)
        prediction = x.dot(self.coefficients)
        self.coefficients -= learning_rate * 1/m * (x.T.dot(prediction - y))
    
    def predict(self, x):
        x = self.initialization(x)
        
        return x.dot(self.coefficients)
    
    def __normal_equation(self, x, y, lam: float):
        features_num = x.shape[1]
        x = self.initialization(x)
        self.x = x
        mat = np.eye((self.degree * features_num) + 1)
        mat[0][0] = 0

        self.coefficients = np.linalg.inv(x.T.dot(x) + lam * mat).dot(x.T.dot(y))
    
    # Overfitting means the error on training data is reducing but it start to increase on test data
    # so the model can't generalize I could check it outside the class by training several times with
    # different number of iterations but this has a lower cost computationally.
    # Example:
    # if iteration = 10000 and step = 100 then the functions measures error each 100 iterations so
    # it measures it 10000/100 = 100 times
    def track_overfitting(self, X_train, X_test, y_train, y_test, learning_rate, iteration: int, step: int):
        features_num = X_train.shape[1]
        # plus 1 is for considering bias
        self.coefficients = np.random.random((self.degree * features_num) + 1)
        
        X_train = self.initialization(X_train)
        X_test = self.initialization(X_test)
        
        while iteration > step:
            for i in range(step):
                self.gradient_descent(X_train, y_train, learning_rate)
                
            self.error_measurement(X_train, X_test, y_train, y_test, step) 
            
            iteration -= step

        for i in range(iteration):
            self.gradient_descent(X_train, y_train, learning_rate)    
        
        self.error_measurement(X_train, X_test, y_train, y_test, step)
        
        self.plot_error(iteration)
           
    def error_measurement(self, X_train, X_test, y_train, y_test, step):
        if len(self.steps) == 0:
            self.steps = np.append(self.steps, 0)
        else:
            self.steps = np.append(self.steps, self.steps[len(self.steps) - 1] + step)
            
        y_train_predicted = X_train.dot(self.coefficients)
        self.train_error_mse = np.append(self.train_error_mse, mean_squared_error(y_train, y_train_predicted))
        self.train_error_mae = np.append(self.train_error_mae, mean_absolute_error(y_train, y_train_predicted))
        self.train_error_rmse = np.append(self.train_error_rmse, sqrt(mean_squared_error(y_train, y_train_predicted)))  
            
        y_test_predicted = X_test.dot(self.coefficients)
        self.test_error_mse = np.append(self.test_error_mse, mean_squared_error(y_test, y_test_predicted))
        self.test_error_mae = np.append(self.test_error_mae, mean_absolute_error(y_test, y_test_predicted))
        self.test_error_rmse = np.append(self.test_error_rmse, sqrt(mean_squared_error(y_test, y_test_predicted)))
    
    def plot_error(self, iteration):
        plt.scatter(self.steps, self.train_error_mse, c='blue')
        plt.scatter(self.steps, self.test_error_mse, c='red')
        plt.title('mean squared error')
        plt.xlabel('steps')
        plt.ylabel('error')
        plt.show()
        
        plt.scatter(self.steps, self.train_error_mae, c='blue')
        plt.scatter(self.steps, self.test_error_mae, c='red')
        plt.title('mean absolute error')
        plt.xlabel('steps')
        plt.ylabel('error')
        plt.show()
                                    
        plt.scatter(self.steps, self.train_error_rmse, c='blue')
        plt.scatter(self.steps, self.test_error_rmse, c='red')
        plt.title('root mean squared error')
        plt.xlabel('steps')
        plt.ylabel('error')
        plt.show() 
        
        print('for the degree {:d} and {:d} iterations the errors for training set are MSE = {}, '
              'MAE = {} and RMSE = {} and the errors for test set are MSE = {}, MAE = {} and RMSE = {}'
              .format(self.degree, iteration, self.train_error_mse[len(self.train_error_mse)-1],
                      self.train_error_mae[len(self.train_error_mae)-1], self.train_error_rmse[len(self.train_error_rmse)-1],
                     self.test_error_mse[len(self.test_error_mse)-1], self.test_error_mae[len(self.test_error_mae)-1],
                      self.test_error_rmse[len(self.test_error_rmse)-1]))

    def plot_fitted(self, X_train, y_train, X_test):
        plt.scatter(X_train, y_train, c='blue')
        plt.scatter(X_test, reg.predict(np.array(X_test).reshape(len(X_test), 1)), c='red')
        plt.show()